In [5]:
from deap import gp
import operator
import math
def square(x):
    return x*x
def absSqrt(x):
    return math.sqrt(abs(x))
pset = gp.PrimitiveSet("nodePrims", arity=1)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(absSqrt, 1)
pset.addPrimitive(square, 1)

In [16]:
from deap import creator, base, tools, algorithms
from random import randint
import numpy as np
import pickle
import itertools
DataDims = 2
MINDIMFUNCS = 1
MAXDIMFUNCS = 10
MINDIMS = DataDims
MAXDIMS = 10
POPSIZE = 50
GENERATIONS = 25
MU = 40
LAMBDA = 30
CXPB = 0.5
MUTPB = 0.3


def genTreeList():
    return [gp.PrimitiveTree(gp.genHalfAndHalf(pset=pset, min_=1,max_=6)) for i in range(randint(MINDIMFUNCS,MAXDIMFUNCS))]
def genFuncList():
    return [genTreeList() for j in range(randint(MINDIMS,MAXDIMS))]

creator.create("FitnessMax", base.Fitness, weights=(1.0,) )
creator.create("Individual", list, fitness=creator.FitnessMax)

# def myMutate(individual):
#     new = [[gp.mutNodeReplacement(x,pset=pset) for x in l] for l in toolbox.clone(individual)]
#     return (tools.initIterate(creator.Individual, lambda: new),)


def getFitMap(ls):
    fitnesses = []
    for l in ls:
        fitnesses.append(l.fitness.values[0])
    return fitnesses

def evolve(evaluator):
    
    
    
    creator.create("FitnessMax", base.Fitness, weights=(1.0,) )
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.Individual, genFuncList)
    
    def myMutate(individual):
        new = [[gp.mutNodeReplacement(x,pset=pset) for x in l] for l in toolbox.clone(individual)]
        new = [[x[0] for x in l] for l in new]
        return (tools.initIterate(creator.Individual, lambda: new),)

    toolbox.register("mate", tools.cxUniform, indpb=0.5)
    toolbox.register("mutate", myMutate)
    toolbox.register("select", tools.selTournament, tournsize = 4)
    toolbox.register("evaluate", evaluator)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(POPSIZE)
    
    

    myStats = tools.Statistics()
    myStats.register("mean", lambda ls: np.mean(getFitMap(ls)))
    myStats.register("min", lambda ls: min(getFitMap(ls)))
    myStats.register("max", lambda ls: max(getFitMap(ls)))
    myStats.register("stdDev", lambda ls: np.std(getFitMap(ls)))

    hallOFame = tools.HallOfFame(5) # hall of fame of size 5


    (finalPop, logbook) = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, GENERATIONS, myStats, halloffame=hallOFame, verbose=True)

    gen = logbook.select("gen")
    fit_maxs = logbook.select("max")
    fit_avgs = logbook.select("mean")

    import matplotlib.pyplot as plt

    fig, ax1 = plt.subplots()
    line1 = ax1.plot(gen, fit_maxs, "b-", label="Maximum Fitness")
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Fitness", color="b")
    for tl in ax1.get_yticklabels():
        tl.set_color("b")

    ax2 = ax1.twinx()
    line2 = ax2.plot(gen, fit_avgs, "r-", label="Average Fitness")
    ax2.set_ylabel("Size", color="r")
    for tl in ax2.get_yticklabels():
        tl.set_color("r")

    lns = line1 + line2
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc="center right")

    plt.show()
    
def flatten(mylist):
    return list(itertools.chain.from_iterable(mylist))
    
def myEval(individual):
    funcs = []
    for dimlist in individual:
        newDimlist = []
        for tree in dimlist:
#             print gp.stringify(tree)
            f = gp.compile(tree, pset)
            newDimlist.append(f)
        funcs.append(newDimlist)
    flattened = flatten(funcs)
    mapped = map(lambda f: f(1), flattened)
    return max(mapped),
    
    
evolve(myEval)

gen	nevals	mean   	min    	max    	stdDev 
0  	50    	12.0987	1.67842	233.558	33.8616
1  	23    	27.0639	4      	233.558	49.6745
2  	27    	52.7762	7.90471	233.558	63.3216
3  	20    	92.0165	11.5403	233.558	77.1398
4  	24    	176.354	57.5442	233.558	82.4405
5  	23    	229.947	57.5442	265.123	28.0427
6  	27    	265.12 	233.558	467.116	76.7934
7  	25    	500.791	233.558	6561   	976.817
8  	25    	1212.91	233.558	6561   	2022.16
9  	23    	3521.36	467.116	6561   	3039.97
10 	22    	6136.84	759.352	6780.24	1531.97
11 	23    	6588.4 	6561   	6780.24	72.5055
12 	23    	6643.21	6561   	6780.24	106.137
13 	27    	6725.91	6561   	6780.24	90.0939
14 	28    	6785.67	6736.53	6910.72	29.4875
15 	24    	6812.86	6780.24	6910.72	56.4995
16 	22    	6858.52	6780.24	6910.72	63.9219
17 	23    	6910.72	6910.72	6910.72	9.09495e-13
18 	24    	6910.72	6910.72	6910.72	9.09495e-13
19 	21    	6984.45	6910.72	7648.05	221.2      
20 	21    	7095.05	6910.72	7648.05	319.275    
21 	18    	7445.28	6910.72	7648.05	329

In [13]:
import mnist
import numpy as np
import theano
training = list(mnist.read(dataset = "training"))
testing = list(mnist.read(dataset = "testing"))

def labelToArray(x):
    blank = [0] * 10
    blank[x] = 1
    return np.array(blank)

trainingimgs = [train[1].astype(theano.config.floatX).ravel() * (1.0/256) for train in training]
traininglabels = [labelToArray(train[0]).astype(theano.config.floatX) for train in training]
testingimgs = [test[1].astype(theano.config.floatX).ravel() * (1.0/256) for test in testing]
testinglabels = [labelToArray(test[0]).astype(theano.config.floatX) * (1.0 / 256) for test in testing]

trainingimgs[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [4]:
import theano.tensor as T
from theano import shared
import theano
from autograd import grad as Grad
import autograd.numpy as np
import autograd.numpy.linalg as LA
import itertools
import typing
import time
import math
from itertools import izip
SPACESPAN = 2.0

def box(x):
    if x > 0 + SPACESPAN / 2.0:
        return -1.0 + x
    elif x < 0 - SPACESPAN / 2.0:
        return 1.0 + x
    else:
        return x
            
def discretize(x, resDenominator):
    if x > 0.0:
        return np.floor(1.0 * x * resDenominator) / resDenominator
    elif x < 0.0:
        return np.floor(1.0 * x * resDenominator) / resDenominator
    else:
        return 0.0
    
def mnistlabelToArray(x):
    blank = [0] * 10
    blank[x] = 1
    return np.array(blank)


def flatten(mylist):
    return list(itertools.chain.from_iterable(mylist))

def newSingleGenConsolidate(arrays):
    bigMatrix = np.array(arrays)
    arraylen = len(arrays)
    initArraylen = len(arrays)
    indices = []
    flattenedIndices = []
    
    irange = range(arraylen)
    
    counter = True
    print "starting a singleGenConsolidate"
    for i in irange:
        if i not in flattenedIndices:
            subgroupArr = np.argwhere(np.all((bigMatrix-bigMatrix[i])==0, axis=0))
            subgroup = subgroupArr.tolist()
            indices.append(subgroup)
            flattenedIndices += subgroup
            arraylen -= subgroupArr.size
            if counter and arraylen/initArraylen < 0.5:
                print "half way done a singleGenConsolidate"
                counter = False
    
    print "finished a singleGenConsolidate"
    return indices
            
            
            
def singleGenConsolidate(arrays):
    indices = []
    flattenedIndices = []
    arraylen = len(arrays)
    initArraylen = len(arrays) * 1.0
    irange = range(arraylen)
    
    counter = True
    print "starting a singleGenConsolidate"
    for i in irange:
        if i not in flattenedIndices:
            subgroup = [i] + [j for j in irange[i + 1:] if np.array_equal(arrays[i], arrays[j])]
            arraylen -= len(subgroup)
            indices.append(subgroup)
            flattenedIndices += subgroup
            if counter and arraylen/initArraylen < 0.5:
                print "half way done a singleGenConsolidate"
                counter = False
    
    print "finished a singleGenConsolidate"
    return indices



def nprelu(x):
    return x * (x > 0)
nprelu = np.vectorize(nprelu)


def getSubV2np(index, inds, complete):
    return complete[inds[index]:inds[index + 1]]

def getSubV3np(index, indinds, inds, complete):
    subInds = inds[indinds[index] : indinds[index + 1] + 1]
    return subInds, complete
    
def singleConsol(tensor, indices):
    return (np.choose(indices, tensor)).sum()

def positionConsolidate(arrays, consolidations):
    return [arrays[sublist[0]] for sublist in consolidations]

def numpyConsolidate(array, consols):
    new = []
    for sublist in consols:
        x = 0.0
        for i in sublist:
            x += array[i]
        new.append(x)

    return new

def npConsolidate(array, consolinds, consols):
    ordered = array.take(consols)
    f = lambda i : ordered[consolinds[i]: consolinds[i + 1]].sum()
    veced = np.vectorize(f)
    return veced(np.arange(consolinds.size))

def generalfeedforward((weights, biases, finalweights, finalbiases), consolMasks, hiddenlayers, branchMultiplier, inp):
    
    for i in range(hiddenlayers):
        inp = np.repeat(inp, branchMultiplier)
        inp = np.multiply(weights[i], inp)
        inp = np.concatenate((inp, np.array([0.0]).astype('float32')))
        inp = inp.take(consolMasks[i]).sum(axis = 1)
        inp = nprelu(np.add(inp, biases[i]))
    
    print inp
    finalout = nprelu(np.dot(inp, finalweights) + finalbiases)
    return finalout


class nnet:
    weights = []
    biases = []
    consolidations = []
    finalweights = []
    finalbiases = []
    weightnum = 0
    hiddenlayers = 0
    
    def __init__(self, resolution, functions, inputdimension, datainp, dataoutp, synapseThreshold):
        self.resDenominator = resolution
        self.funcs = functions
        self.dimensions = len(self.funcs)
        self.branchMultiplier = len(flatten(self.funcs))
        self.TBranchMult = shared(np.array([self.branchMultiplier]))
#         self.dataset = [shared(dat.astype(theano.config.floatX)) for dat in datainp]
#         self.datalabels = [shared(outp.astype(theano.config.floatX)) for outp in dataoutp]
        self.dataSample = datainp[0] #numpy
        self.dataOutSample = dataoutp[0]
        self.inputdimension = inputdimension
        
        if self.dataSample.size ** (1.0/ inputdimension) > self.resDenominator * 2.0:
            print "resDenominator may be too small for effective learning"
            
        self.threshold = synapseThreshold

    def applyFuncs(self, narray): #checked
        boxvec = np.vectorize(lambda x: discretize(box(x), self.resDenominator))
        displaced = []
        
        for dim in range(self.dimensions):
            for f in self.funcs[dim]:
                
                zeroes = np.zeros_like(narray)
                displacement = f(narray[dim]) % SPACESPAN
                zeroes[dim] = displacement
                
                newArray = np.add(narray, zeroes)
                newArrayBoxed = boxvec(newArray)
                displaced.append(newArrayBoxed)
                
        
        return displaced
        

    def applyFuncsMult(self, narrays): 
#         print "narrays: " + str(narrays)
        return flatten([self.applyFuncs(x) for x in narrays])
    
    def locate(self, sample):  #only 1 or 2 implemented
        insize = sample.size
        sample = np.ravel(sample) 
        tensorFrame = [0] * self.dimensions
        located = [0] * insize
        
        boxvec = np.vectorize(lambda x: discretize(box(x), self.resDenominator))
        
        #self.inputdimension is the dimension the input should be represented in
        
        if self.inputdimension == 1:
            
            for i in range(insize):
                myTens = tensorFrame[:]
                myTens[0] = (2.0 * i) / insize - 1.0
                located[i] = boxvec(np.array(myTens))
        
        if self.inputdimension == 2:
            located = []
            factorPairs = [(i,(insize / i)) for i in range(1, int(math.floor(insize**0.5))) if insize % i == 0]
            pair = factorPairs[-1]
            for i in range(pair[0]):
                for j in range(pair[1]):
                    myTens = tensorFrame[:]
                    myTens[0] = 2.0 * i/pair[0] - 1.0
                    myTens[1] = 2.0 * j/pair[1] - 1.0
                    located.append(boxvec(np.array(myTens)))
                            
        return located
        
    def genConsolidate(self): #finds consolidation list from located self.dataSample
        consols = []
#         print len(self.dataSample.ravel())
        located = self.locate(self.dataSample)
#         print len(located)
#         print type(located[0])
        print "running initial singleGenConsolidate"
        located = self.applyFuncsMult(located)
        consols.append(singleGenConsolidate(located))
        located = positionConsolidate(located, consols[-1]) 

#         print located
        print "starting to loop"
        while len(flatten(flatten(consols))) + len(flatten(consols[-1])) * self.branchMultiplier < self.threshold:
            located = self.applyFuncsMult(located)
            testStart = time.time()
            consols.append(singleGenConsolidate(located))
            print "first version took: " + str(time.time() - testStart)
#             testStart = time.time()
#             x = newSingleGenConsolidate(located)
#             print "new version took: " + str(time.time() - testStart)
            
            located = positionConsolidate(located, consols[-1]) 
            print "looped"
            
        self.consolidations = consols
        self.consolMasks = []
        for layerConsols in self.consolidations:
#             print "initial " + str(layerConsols)
            maxInLength = max(map(len, layerConsols))
            inLength = len(flatten(layerConsols))
            extended = map(lambda l: l + [inLength] * (maxInLength - len(l)), layerConsols)
#             print "extended: " + str(extended)
            self.consolMasks.append(np.array(extended).astype('int32'))
        return consols
    
    def genWeights(self):
        myWeightNum = 0
        self.hiddenlayers = 0
        self.weights = []
        self.biases = []
        self.finalweights = []
        self.finalbiases = []
        for consol in self.consolidations:
            weightVec = (np.random.rand(len(flatten(consol))) * 0.05).astype(theano.config.floatX).tolist()
            biasVec   = (np.random.rand(len(consol)) * 0.05).astype(theano.config.floatX).tolist()
            self.weights.append(weightVec)
            self.biases.append(biasVec)
            myWeightNum += len(weightVec)

            #convert to shared data type for later speed
            self.hiddenlayers += 1
        
        outsize = len(self.dataOutSample)
        
        self.finalweights = np.random.rand(len(self.consolidations[-1]),outsize).astype('float32')        #final interconnected layer for output
        self.finalbiases = np.random.rand(outsize).astype('float32')

        myWeightNum += len(self.consolidations[-1]) * outsize
#         self.weightNum = myWeightNum
        
    def feedforward(self, inp):
        def foo(x):
            if x > 0.0:
                return x
            else:
                return 0.0
        relu = np.vectorize(foo)
        for i in range(self.hiddenlayers):
            inp = np.repeat(inp, self.branchMultiplier)
#             print "repeated: " + str(inp)
            inp = np.multiply(self.weights[i], inp)
#             print "weighted: " + str(inp)
            inp = numpyConsolidate(inp, self.consolidations[i])
#             print "consolidated: " + str(inp)
            inp = nprelu(np.add(inp, self.biases[i]))
            print "biased and tanned: " + str(inp)
        
#         print "before finals: " + str(inp)
#         print "finalweights: " + str(self.finalweights)
#         print "finalbiases: " + str(self.finalbiases)
        finalout = nprelu(np.dot(inp, self.finalweights) + self.finalbiases)
#         print "final: " + str(finalout)
        return finalout

    def nfeedforward(self, inp):
        relu = np.vectorize(lambda x: x * (x > 0))
        for i in range(self.hiddenlayers):
            inp = np.repeat(inp, self.branchMultiplier)
            inp = np.multiply(self.weights[i], inp)
            inp = np.concatenate((inp, np.array([0.0]).astype('float32')))
            inp = inp.take(self.consolMasks[i]).sum(axis = 1)
            inp = relu(np.add(inp, self.biases[i]))
        return relu(np.dot(inp, self.finalweights) + self.finalbiases)
    
    def test(inputs, outputs):
        def foo(x):
            if x > 0.5:
                return 1.0
            return 0.0

        binarize = np.vectorize(foo)
        
        errors = [LA.norm(outp - binarize(self.feedforward(inp))) for inp,outp in izip(inputs,outputs)]
        return sum(errors)/len(inputs) * 100.0
        
    
    def numpytrain(self, alpha, epochs, batchsize, inputlist, outputlist, verbose, testdatainputs, testdatalabels):
        print "starting training"
        
        def error((weights, biases, finalweights, finalbiases), inp, outp):
            return LA.norm(outp - generalfeedforward((weights,biases,finalweights,finalbiases), 
                                                     self.consolMasks, self.hiddenlayers, self.branchMultiplier, inp))
        error_grad = Grad(error)
        
        inlen = len(inputlist)
        outlen = len(outputlist)

        if inlen != outlen:
            Exception("number of input vectors ("+str(inlen)+") not equal not number of ouptut vectors ("+str(outlen)+")")
            
        if alpha == 0.0:
            raise("why is alpha zero?")
            
        if type(batchsize) != int:
            raise("Why is batchsize not an int?")
        
        if batchsize == 0 or batchsize > inlen:
            raise("batchsize of "+str(batchsize)+"is not allowed. Note than inputveclist has length "+str(inlen))
            
        #just some error checks
        
        randindexlist = range(inlen)
        start = time.time()
        avgpercenttesterror = "No test set"
        avgpercenttesterrorlist = []
        
        #using stochastic gradient descent training method
        for epoch in xrange(epochs):
            #random.shuffle(randindexlist)
            #for batch in range(inlen/batchsize):
            
#                 low = batch * batchsize
#                 upper = (batch + 1) * batchsize
#                 if upper > inlen: upper = inlen 
                
#                 total_Egradweights = None
#                 total_Egradbiases = None
                
#                 for i in randindexlist[low:upper]:
            for inp,outp in izip(inputlist,outputlist):
                Egradws, Egradbs, Egadfws, Egradfbs = error_grad((self.weights, self.biases, 
                                                       self.finalweights, self.finalbiases), inp, outp) #formerly used indexes from randindexlist
#                     if total_Egradweights:
#                         total_Egradweights = [x + y for x,y in izip(Egradweights,total_Egradweights)]
#                         total_Egradbiases = [x + y for x,y in izip(Egradbiases,total_Egradbiases)]
#                     else:
#                         total_Egradweights = Egradweights
#                         total_Egradbiases = Egradbiases
                
#                 total_Egradweights = [x / (batchsize * 1.0) for x in total_Egradweights]
#                 total_Egradbiases = [x / (batchsize * 1.0) for x in total_Egradbiases]
                
#                 self.weights = [curr-grad*alpha for curr, grad in izip(self.weights, total_Egradweights)]
#                 self.biases = [curr-grad*alpha for curr, grad in izip(self.biases, total_Egradbiases)]
                self.weights = [curr-grad*alpha for curr, grad in izip(self.weights, Egradws)]
                self.biases = [curr-grad*alpha for curr, grad in izip(self.biases, Egradbs)]
                self.finalweights = self.finalweights - alpha * Egradfws
                self.finalbiases = self.finalbiases - alpha * Egradfbs
            
            if len(testinputlist) == len(testoutputlist) and len(testinputlist) != 0:
                avgpercenttesterror = self.test(testinputlist, testoutputlist)
                avgpercenttesterrorlist.append(avgpercenttesterror)
            else:
                print "bad test set"
            
            elapsed = time.time() - start
            start = time.time()
            if verbose:
                print "epoch: " + str(epoch) + "\t percenterror: " + str(avgpercenttesterror) + "% \t time elapsed this epoch: " + str(elapsed) + "s"
            
        return (avgpercenttesterror, avgpercenttesterrorlist)

# singleGenConsolidate([np.array([0,0,0]), np.array([0,1,2,3]), np.array([0,0,0])])
testnet = nnet(10, [[lambda x: x + 1], [lambda x: x * 2]], 1, [np.array([0,1,2,3])], [np.array([1])], 100)
testnet.genConsolidate()
testnet.genWeights()
numpyConsolidate(np.array([1,2,3]), [[0,2], [1]])
print testnet.nfeedforward(np.array([0,1,2,3]))
print testnet.feedforward(np.array([0,1,2,3]))
# print mynet.consolidations
        
        
        


running initial singleGenConsolidate
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
starting to loop
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000173807144165
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.0001540184021
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000154972076416
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000158071517944
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000166177749634
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version to

In [68]:
import theano
from theano import shared,config,function
import itertools
import theano.tensor as T
import numpy as np
from collections import OrderedDict

# theano.config.exception_verbosity = "high"

def flatten(mylist):
    return list(itertools.chain.from_iterable(mylist))

class V2IndexedShared:
    def __init__(self, atype):
        self.myType = atype
    
    def fromList(self,D2List):
        self.complete = shared(np.array(flatten(D2List)).astype(self.myType))
        self.length = shared(len(D2List))
        inds = [0]
        for sublist in D2List:
            inds.append(len(sublist) + inds[-1])
        self.inds = shared(np.array(inds).astype('int32'))
        return self

    
class V3IndexedShared:
    def __init__(self,D3List, atype):
        self.complete = shared(np.array(flatten(flatten(D3List))).astype(atype))
#         self.complete = np.array(flatten(flatten(D3List))).astype(atype)

        self.myType = atype
        self.length = shared(len(D3List))
        D2Inds = [0]
        D3Inds = [0]
        
        for sublistOLists in D3List:
            
            for sublist in sublistOLists:
                D2Inds.append(len(sublist) + D2Inds[-1])
                
            D3Inds.append(len(sublistOLists) + D3Inds[-1])
            
                
        self.D2Inds = shared(np.array(D2Inds).astype('int32'))
        self.D3Inds = shared(np.array(D3Inds).astype('int32'))
        
#         self.D2Inds = np.array(D2Inds).astype('int32')
#         self.D3Inds = np.array(D3Inds).astype('int32')
#         print D3Inds
#         print D2Inds
#         print np.array(flatten(flatten(D3List)))
    

def relu(x):
    return T.maximum(x, 0)


def getSubV2(index, inds, complete):
    return complete[inds[index]:inds[index + 1]]

def getSubV2Check():
    tind = T.iscalar('ind')
    tinds = T.ivector('inds')
    tcomp = T.vector('complete')
    
    nind = np.asscalar(np.array([0]).astype('int32'))
    ninds = np.array([0,2,4]).astype('int32')
    comp = np.array([0,1,2,3,4]).astype(theano.config.floatX)
    
    f = theano.function([tind, tinds, tcomp], getSubV2(tind, tinds, tcomp))
    print f(nind,ninds,comp)
getSubV2Check()
def getV2Length(inds):
    return inds.size -1

def getSubV3(index, indinds, inds, complete):
    subInds = inds[indinds[index] : indinds[index + 1] + 1]
    return subInds, complete

def getSubV3Check():
    D3List = [[[0,1,2,], [3,4,5]], [[6,7,8], [9,10,11]]]
    consolcomplete = [0,1,2,3,4,5,6,7,8,9,10,11]
    consolinds = [0,3,6,9,12]
    consolindinds = [0,2,4]
    print getSubV3(1, consolindinds, consolinds, consolcomplete)
    print consolinds[2:5]
    t = V3IndexedShared(D3List, 'int32')
    
# getSubV3Check()
    
def singleConsol(tensor, indices):
    return (T.choose(indices, tensor)).sum()

def consolidateTensor(tensor, consolinds, consolcomplete):
    irange = T.arange(getV2Length(consolinds))
    consolidated, updates = theano.scan(fn = lambda ind, tens, coninds, concomp: singleConsol(tens, getSubV2(ind, coninds, concomp)),
                                        sequences = irange,
                                        non_sequences=[tensor, consolinds, consolcomplete])
    return consolidated

def consolidateTensorCheck():
    x = T.vector('x')
    inds = T.ivector('inds')
    complete = T.ivector('complete')
    y = consolidateTensor(x,inds,complete)
    f = theano.function([x,inds,complete], y)
    print f(np.array([1.0,2.0,4.0,8.0]).astype(theano.config.floatX), np.array([0,3,4]).astype('int32'), np.array([1,2,3,0]).astype('int32'))


    
from random import shuffle

class tnnet:
    def __init__(self, resolution, functions, inputdimension, traindatainps, traindataoutps, testdatainps, testdataoutps, synapseThreshold, net = None):
        
        mynet = None
        if net == None:
            mynet = nnet(resolution, functions, inputdimension, traindatainps, traindataoutps, synapseThreshold)
            mynet.genConsolidate()
            mynet.genWeights()
        else:
            mynet = net
        self.nnet = mynet
        

        self.sharedTrainingInps = shared(np.array(traindatainps).astype(theano.config.floatX))
        self.sharedTrainingOutps = shared(np.array(traindataoutps).astype(theano.config.floatX))
        self.sharedTestingInps = shared(np.array(testdatainps).astype(theano.config.floatX))
        self.sharedTestingOutps = shared(np.array(testdataoutps).astype(theano.config.floatX))


#         print "weights: " + str(mynet.weights)
#         print "biases: " + str(mynet.biases)
#         print mynet.feedforward(np.array([0,1,2,3]).astype(theano.config.floatX))
#         inp_lengths = [len(consol) for consol in mynet.consolidations]
#         inp_lengths = [datainps[0].size] + inp_lengths
#         self.inp_lengths = np.array(inp_lengths).astype('int32')
#         print "inp_lengths: " + str(inp_lengths)
#         print "dinp length: " + str(datainps[0].size)
#         print "consolidations: " + str(mynet.consolidations)
        
#         self.maxInp = np.amax(inp_lengths)
#         self.maxPad = shared(np.array([0] * (np.amax(inp_lengths) - min(inp_lengths))).astype(theano.config.floatX))
#         self.maxZeroes = shared(np.array([0.0] * self.maxInp).astype(theano.config.floatX))
        
#         initialPaddingLength = np.amax(inp_lengths) - datainps[0].size
#         self.initialPadding = shared(np.array([0.0] * initialPaddingLength).astype(theano.config.floatX))
#         self.inp_lengths = shared(self.inp_lengths)
        
        tweights = V2IndexedShared(theano.config.floatX)
        tweights.fromList(mynet.weights)
        self.weights = map(lambda x: shared(np.array(x).astype(theano.config.floatX)), mynet.weights)
        
        tbiases = V2IndexedShared(theano.config.floatX)
        tbiases.fromList(mynet.biases)
#         print tbiases.complete.get_value()
        self.biases = map(lambda x: shared(np.array(x).astype(theano.config.floatX)), mynet.biases)
    
        tcons = V3IndexedShared(mynet.consolidations, 'int32')
#         print "final weights: " + str(mynet.finalweights)
        tfws = shared((mynet.finalweights).astype(theano.config.floatX))
        tfbs = shared((mynet.finalbiases).astype(theano.config.floatX))
        self.weightnum = mynet.weightnum
        self.hiddenlayers = mynet.hiddenlayers
        
        self.consolMasks = []
        self.dEdWsinverseConsols = []
        self.oneMultipliers = []
        for layerConsols in mynet.consolidations:
#             print "initial " + str(layerConsols)
            maxInLength = max(map(len, layerConsols))
            inLength = len(flatten(layerConsols))
            extended = map(lambda l: l + [inLength] * (maxInLength - len(l)), layerConsols)
#             print "extended: " + str(extended)
            self.consolMasks.append(shared(np.array(extended).astype('int32')))
            self.oneMultipliers.append(shared(np.array([1.0] * maxInLength).astype(theano.config.floatX)))
            
            flattened = flatten(layerConsols)
            frame = [0] * len(flattened)
            for i in range(len(layerConsols)):
                for sub in layerConsols[i]:
                    frame[sub] = i
            
            self.dEdWsinverseConsols.append(shared(np.array(frame).astype('int32')))
        
#         print self.nnet.consolidations[0]
#         print self.dEdWsinverseConsols[0].get_value()
            
        
        
        inp = T.fvector('inp')
#         paddedInp = T.concatenate([inp, self.initialPadding])
        actualOutp = T.fvector('actualOutp')
        alpha = T.fscalar('alpha')
        
        self.consolidations = map(lambda x: V2IndexedShared('int32').fromList(x), self.nnet.consolidations)
        
        self.winds = tweights.inds
        self.ws = tweights.complete
        self.binds = tbiases.inds
        self.bs = tbiases.complete
        self.consindinds = tcons.D3Inds
        self.consinds = tcons.D2Inds
        self.cons = tcons.complete
        self.bMult = shared(np.asscalar(np.array([mynet.branchMultiplier]).astype('int32')))
        self.fws = tfws
        self.fbs = tfbs
#         print "weights: " + '\n'.join(map(str, self.nnet.weights))
# #         print self.nnet.biases
#         print "inverse Consols: " + '\n'.join(map(lambda x: str(x.get_value().tolist()), self.dEdWsinverseConsols))
#         print "consols: " + '\n'.join(map(str, self.nnet.consolidations))
        self.zeropad = shared(np.array([0.0]).astype(theano.config.floatX))
        
        self.bMultOnes = shared(np.array([1.0] * self.nnet.branchMultiplier).astype(theano.config.floatX))
        
        self.params = self.weights + self.biases + [self.fws, self.fbs]
#         self.constparams = [self.consindinds, self.consinds, self.cons, self.winds, self.binds, self.bMult]
        
#         def singleCons(consInds, tens):
#             asum, _ = theano.scan(fn = lambda i, tensor: tensor[i],
#                                  sequences = consInds,
#                                  non_sequences = tens)
#             return asum.sum()
        
        
#         def layer(ind, inpLength, inp):
#             weights = getSubV2(ind, self.winds, self.ws)
#             biases = getSubV2(ind, self.binds, self.bs)
#             consolinds, consols = getSubV3(ind, self.consindinds, self.consinds, self.cons)
#             inp = T.repeat(inp[0:inpLength], self.bMult)
#             inp = weights * inp
            
#             #consolidation code
#             numConsolidatedRange = T.arange(consolinds.size - 1)
#             inp, _ = theano.scan(fn = lambda ind, x, coninds, concomp: singleCons(getSubV2(ind, coninds, concomp), x),
#                                                 sequences = numConsolidatedRange,
#                                                 non_sequences=[inp, consolinds, consols])
#             inp = inp + biases
#             inp = relu(inp)
#             padding = self.maxPad[0 : self.maxInp - inp.size]
#             inp = T.concatenate([inp, padding])
#             return inp


        def gradLayer(ind, nextdEdInputs, layerOutput): #returns (dEdWs, dEdBs, newdEdInputs)
            expandedLayerOutput = T.repeat(layerOutput, self.bMult)
            inverse = self.dEdWsinverseConsols[ind]
            inverseddEdInputs = nextdEdInputs.take(inverse)
            dEdWs = expandedLayerOutput * inverseddEdInputs
            dEdBs = nextdEdInputs
            #upt to here correct
            
            dOutsdIns = T.gt(layerOutput, T.zeros_like(layerOutput))
            dOutsdIns = T.cast(dOutsdIns, 'float32')
            
            dEdConnections = self.weights[ind] * inverseddEdInputs
            dEdConnectionGroups = dEdConnections.reshape((dEdConnections.size // self.bMult, self.bMult))
            dEdOuts = T.dot(dEdConnectionGroups, self.bMultOnes) #could need to be axis 1
            
            newdEdInputs = dEdOuts * dOutsdIns
            
            return (dEdWs, dEdBs, newdEdInputs)
        
        def intermediateLoop(z):
            ret = []
            for i in range(self.hiddenlayers):
                z = layer(i, z)
                ret.append(z.copy())
            
            return ret
        
        def myGrad(theta, actualOutp):
            outputs = intermediateLoop(theta)
            outputs = [theta] + outputs
            last = T.dot(outputs[-1], self.fws)
            last = last + self.fbs
            last = theano.tensor.nnet.relu(last)
            
            
            
            finaldOutdIn = T.gt(last, T.zeros_like(last))
            finaldOutdIn = T.cast(finaldOutdIn, 'float32')
            
            difference = last - actualOutp
            err = T.dot(difference, difference)
            
            initdEdIn = finaldOutdIn * (last - actualOutp)
            dEdFbs = initdEdIn   
            
            
            initdEdInshuffled = initdEdIn.dimshuffle(('x',0))
            finalStructuredLayerOut = outputs[-1]
            finalStructuredLayerOutShuffled = finalStructuredLayerOut.dimshuffle((0,'x'))
            
            dEdFws = finalStructuredLayerOutShuffled * initdEdInshuffled
            
            
            finalStructuredLayerdOutdIn = T.gt(finalStructuredLayerOut, T.zeros_like(finalStructuredLayerOut))
            finalStructuredLayerdOutdIn = T.cast(finalStructuredLayerdOutdIn, 'float32')
            
            finalStructuredLayerdEdOut = T.dot(initdEdIn, self.fws.T)
            
            finalStructuredLayerdEdIn = finalStructuredLayerdOutdIn * finalStructuredLayerdEdOut
            #up to here is correct

            dEdWs, dEdBs = gradLoop(finalStructuredLayerdEdIn, outputs)
            
            return err, list(reversed(dEdWs)), list(reversed(dEdBs)), dEdFws, dEdFbs #, list(reversed(dEdIns)),
            
        def gradLoop(dEdIn, layerOutputs):
            dEdWs = []
            dEdBs = []
            for k in reversed(range(self.hiddenlayers)):
                newdEdWs, newdEdBs, dEdIn = gradLayer(k, dEdIn, layerOutputs[k])
                dEdWs.append(newdEdWs)
                dEdBs.append(newdEdBs)
            
            return dEdWs, dEdBs
            
            
            
            

        def layer(ind, x):
#             print "in layer"
#             consolinds = self.consolidations[ind].inds
#             consols = self.consolidations[ind].complete
            
            x = T.repeat(x, self.bMult) * self.weights[ind]
            x = T.concatenate([x, T.zeros_like(x)])
            x = (x.take(self.consolMasks[ind]).astype(theano.config.floatX)) #.sum(axis = 1)
            x = T.dot(x, self.oneMultipliers[ind])
# #             inp, _ = theano.map(fn = lambda i: inp[i],
# #                                sequences = consols)
# #             #consolidation code
#             r = T.arange(consolinds.size - 1).astype(theano.config.floatX)
#             for i in xrange(len(self.nnet.consolidations[ind])):
#                 r = T.set_subtensor(r[i], x[consolinds[i]:consolinds[i + 1]].sum())
#             numConsolidatedRange = T.arange(consolinds.size - 1).astype('int32')
# #             inp2, _ = theano.map(fn = lambda i: inp1[consolinds[i]: consolinds[i + 1]].sum(),
# #                                                 sequences = numConsolidatedRange)
#             r, _ = theano.map(fn = lambda i, x: x[consolinds[i]: consolinds[i+1]].sum(),
#                                                 sequences = numConsolidatedRange,
#                                                 non_sequences=[x])
            return theano.tensor.nnet.relu(x + self.biases[ind])
#             return r
            
#        = layer(0, inp)
        final = T.fvector()
        def loop(z):
            ret = T.fvector()
            for i in range(self.hiddenlayers):
                z = layer(i, z)
                ret = z
            return ret
        finals = loop(inp)
        final = T.dot(finals, self.fws)
        final = final + self.fbs
        final = theano.tensor.nnet.relu(final)
        
        diff = final - actualOutp
        squared = 0.5 * T.dot(diff, diff)
        self.classify = theano.function([inp], final)
#         print self.classify(np.array([0,1,2,3]).astype(theano.config.floatX))

        index = T.iscalar()

        squaredError, wgrads, bgrads, fwsgrads, fbsgrads = myGrad(inp, actualOutp)
#         mygradients = myGrad(inp, actualOutp)
        self.realGrad = theano.function(inputs = [inp, actualOutp], outputs = T.grad(squared, self.weights))
        self.mygrads = theano.function(inputs = [inp, actualOutp], outputs = wgrads)
        gradients = wgrads + bgrads + [fwsgrads, fbsgrads]
#         realgrads = T.grad(squared, self.params)
#         realUpdates = OrderedDict((p, p - alpha * g) for p, g in zip(self.params, realgrads))
        param_Updates = OrderedDict((p, p - alpha * g) for p, g in zip(self.params, gradients))
        
        self.intermediates = theano.function([inp], intermediateLoop(inp))
        
        self.train = theano.function(inputs = [index, alpha],
                                                                    outputs = squaredError,
                                                                    updates = param_Updates,
                                                                    givens = {
                                                                        inp : self.sharedTrainingInps[index],
                                                                        actualOutp: self.sharedTrainingOutps[index]
                                                                    })
#         self.realtrain = theano.function(inputs = [index, alpha],
#                                                                     outputs = squared,
#                                                                     updates = realUpdates,
#                                                                     givens = {
#                                                                         inp : self.sharedTrainingInps[index],
#                                                                         actualOutp: self.sharedTrainingOutps[index]
#                                                                     })
        self.test = theano.function(inputs = [index],
            outputs = final,
            givens = {
                inp : self.sharedTestingInps[index]
            })
    
    def descend(self, alpha, epochs, trainingInps, trainingOutps, testingInps, testingOutps, verbose):
        training = zip(trainingInps, trainingOutps)
        testing = zip(testingInps, testingOutps)
        testLen = len(testing)
        trainLen = len(trainingInps)
        indorder = range(trainLen)
        testingAccuracies = []
        
        for i in range(epochs):
            print "starting epoch..."
            start = time.time()
            shuffle(indorder)
            for j in indorder:
                self.train(j, alpha)
            
            testingAccuracy = 0.0
            for j in range(len(testingOutps)):
                if np.argmax(self.test(j)) == np.argmax(testingOutps[j]):
                    testingAccuracy += 1.0
            # for (testInp, testOutp) in testing:
            #     if np.argmax(self.classify(testInp)) == np.argmax(testOutp):
            #         testingAccuracy += 1.0
                    
            percentTestingAccuracy = testingAccuracy / testLen * 100.0
            testingAccuracies.append(percentTestingAccuracy)
            end = time.time()
            if verbose:
                print "epoch " + str(i + 1) + " -- testing accuracy : " + str(percentTestingAccuracy) + " duration: " + str(end - start) + "s"
        
        return max(testingAccuracies), testingAccuracies
            
        
tn = tnnet(10, [[lambda x: x + 1], [lambda x: x * 2]], 1, [np.array([0,1,2,3])], [np.array([1,2,3,4])],  [np.array([1,2,3,4])],  [np.array([1,2,3,4])], 100)
print tn.nnet.feedforward(np.array([0,1,2,3]))
print tn.intermediates(np.array([0,1,2,3]).astype(theano.config.floatX))
# print tn.nnet.weights
# print tn.nnet.biases
x = np.array([0,1,2,3]).astype(theano.config.floatX)
classed = tn.classify(np.array([0,1,2,3]).astype(theano.config.floatX))
# print '\n'.join(map(str,classed))
# diff = x * 0.5 - classed
# print np.dot(diff, diff)
# print tn.classify(np.array([0,1,2,3]).astype(theano.config.floatX))
print '\n'.join(map(str, tn.realGrad(x, x * 0.33)))
print '\n'.join(map(str, tn.mygrads(x, x*0.33)))
map(type, tn.mygrads(x, x * 0.33))

[ 0.  1.]
running initial singleGenConsolidate
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
starting to loop
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000130176544189
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.00011682510376
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000127077102661
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000118970870972
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first version took: 0.000164031982422
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
first

[numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray]

In [66]:
import deap.gp as gp
import time
import sys
sys.setrecursionlimit(1500)
weights = []
biases = []
branching = 0
consolidations = []
fws = []
fbs = []
hiddenlayers = 0
weightnum = 0
branching = 0
def mnistevaluate(individual):
    print "evaluating"
    funcs = []
    for dimlist in individual:
        newDimlist = []
        for tree in dimlist:
#             print gp.stringify(tree)
            f = gp.compile(tree, pset)
            newDimlist.append(f)
        funcs.append(newDimlist)
    
    res = 20
    tnet = tnnet(res, funcs, 2, trainingimgs, traininglabels, testingimgs, testinglabels, 200000)
    mynet = tnet.nnet
    weights = mynet.weights
    biases = mynet.biases
    branching = mynet.branchMultiplier
    consolidations = mynet.consolidations
    fws = mynet.finalweights
    fbs = mynet.finalbiases
    hiddenlayers = mynet.hiddenlayers
    weightnum = mynet.weightnum
    print "branchmultiplier: " + str(mynet.branchMultiplier)
    print "weights length: " + str(len(flatten(mynet.weights)))
#     print "consolidations lengths" + str(map(len, mynet.consolidations))
    layerDensities = []
    prevlayerLength = trainingimgs[0].size
    for layer in mynet.weights:
        layerLength = len(layer)
        branched = prevlayerLength * mynet.branchMultiplier
        layerDensities.append((1.0 * branched) / layerLength)
        prevlayerLength = layerLength
    print "average number of connections per neuron by layer: " + str(layerDensities)
    print "spacial dimension: " + str(len(funcs))
    print "branching by layer: " + str(map(len, funcs))
    print "beginning training"
    testStart = time.time()
    x = tnet.train(0, 0.05)
    print "trained in: " + str(time.time() - testStart)
    print x
    print tnet.classify(trainingimgs[0])
#     testStart = time.time()
#     x = tnet.realtrain(0, 0.05)
    print "autograd trained in: " + str(time.time() - testStart)
    
    avgpercenttestAccuracy, avgpercenttestAccuracylist = tnet.descend(0.02, 5, trainingimgs,traininglabels, testingimgs, testinglabels, True)
    return avgpercenttestAccuracy


evolve(mnistevaluate)

evaluating
running initial singleGenConsolidate
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
starting to loop
branchmultiplier: 33
weights length: 25872
average number of connections per neuron by layer: [1.0]
spacial dimension: 7
branching by layer: [3, 10, 4, 4, 2, 5, 5]
beginning training
trained in: 0.00302386283875
115600.453125
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
autograd trained in: 0.00561690330505
starting epoch...
epoch 1 -- testing accuracy : 9.8 duration: 161.044973135s
starting epoch...


KeyboardInterrupt: 

In [65]:
type(fbs)

list

In [18]:
def tnetFromParams(weights, biases, fws, fbs, consolidations, hiddenlayers, weightnum, branching):
    frame = nnet(10, [[lambda x: x + 1], [lambda x: x * 2]], 1, [np.array([0,1,2,3])], [np.array([1])], 100)
    frame.weights = weights
    frame.biases = biases
    frame.finalweights = np.array(fws).astype(theano.config.floatX)
    frame.finalbiases = np.array(fbs).astype(theano.config.floatX)
    frame.consolidations = consolidations
    frame.hiddenlayers = hiddenlayers
    frame.weightnum = weightnum
    frame.branchingmultiplier = branching
    print "starting theano net instance..."
    mytnet = tnnet(10, [[lambda x: x + 1], [lambda x: x * 2]], 1, trainingimgs, traininglabels, testingimgs, testinglabels, 5000, net = frame)
    testStart = time.time()
    x = mytnet.classify(trainingimgs[0])
    print "classified in " + str(time.time() - testStart)
    testStart = time.time()
    x = frame.feedforward(trainingimgs[0])
    print "numpy classified in " + str(time.time() - testStart)

tnetFromParams(weights, biases, fws, fbs, consolidations, hiddenlayers, weightnum, branching)

starting theano net instance...


UnusedInputError: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 0 is not part of the computational graph needed to compute the outputs: inp.
To make this error into a warning, you can pass the parameter on_unused_input='warn' to theano.function. To disable it completely, use on_unused_input='ignore'.

In [1]:
import theano.tensor.nnet as nnet
import theano.tensor as T
import theano
import numpy as np
x = T.dvector()
y = T.dscalar()
def layer(x, w):
    b = np.array([1], dtype=theano.config.floatX)
    new_x = T.concatenate([x, b])
    m = T.dot(w.T, new_x) #theta1: 3x3 * x: 3x1 = 3x1 ;;; theta2: 1x4 * 4x1
    h = nnet.sigmoid(m)
    return h
def grad_desc(cost, theta):
    alpha = 0.1 #learning rate
    return theta - (alpha * T.grad(cost, wrt=theta))
theta1 = theano.shared(np.array(np.random.rand(3,3), dtype=theano.config.floatX)) # randomly initialize
theta2 = theano.shared(np.array(np.random.rand(4,1), dtype=theano.config.floatX))
hid1 = layer(x, theta1) #hidden layer
out1 = T.sum(layer(hid1, theta2)) #output layer
fc = (out1 - y)**2 #cost expression


cost = theano.function(inputs=[x, y], outputs=fc, updates=[
        (theta1, grad_desc(fc, theta1)),
        (theta2, grad_desc(fc, theta2))])
run_forward = theano.function(inputs=[x], outputs=out1)
inputs = np.array([[0,1],[1,0],[1,1],[0,0]]).reshape(4,2) #training data X
exp_y = np.array([1, 1, 0, 0]) #training data Y
cur_cost = 0
for i in range(10000):
    for k in range(len(inputs)):
        cur_cost = cost(inputs[k], exp_y[k]) #call our Theano-compiled cost function, it will auto update weights
    if i % 500 == 0: #only print the cost every 500 epochs/iterations (to save space)
        print('Cost: %s' % (cur_cost,))
        


#Training done! Let's test it out
print(run_forward([0,1]))
print(run_forward([1,1]))
print(run_forward([1,0]))
print(run_forward([0,0]))




Cost: 0.752302922028
Cost: 0.234828531853
Cost: 0.217997165391
Cost: 0.176582778796
Cost: 0.0941889559431
Cost: 0.0547008715418
Cost: 0.0227962112237
Cost: 0.0103721161976
Cost: 0.00631427914256
Cost: 0.00444667437915
Cost: 0.00339910571615
Cost: 0.00273639533448
Cost: 0.00228238999109
Cost: 0.00195317405859
Cost: 0.00170428799655
Cost: 0.00150985199376
Cost: 0.00135402741352
Cost: 0.00122653983734
Cost: 0.00112027524709
Cost: 0.00103046853549
0.971243725343
0.0324757818475
0.97118115104
0.0308740290756


Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN not available)


In [1]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.332850 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu


Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN not available)


In [48]:
x = T.fvector()
y = x
y = T.set_subtensor(y[3], 1)
out = y
f = theano.function([x], out)
f(np.array(range(5)).astype(theano.config.floatX))

array([ 0.,  1.,  2.,  1.,  4.], dtype=float32)

In [61]:
bs = np.array([0.0,0.0,0.0])
ws = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])
dEdIns = np.array([1,2,3,4,5,6])
prevOut = np.array([2,2,2,2,2])
bMult = 2
bMultOnes = np.array([1.0,1.0])

layerConsols = [[0], [1,2], [3,4], [7,8], [5,6], [9]]
flattened = flatten(layerConsols)
inverseConsols = [0] * len(flattened)
for i in range(len(layerConsols)):
    for sub in layerConsols[i]:
        inverseConsols[sub] = i

inverseConsols = np.array([0,1,1,2,2,4,4,3,3,5])

    
def gradLayer(nextdEdInputs, layerOutput): #returns (dEdWs, dEdBs, newdEdInputs)
        expandedLayerOutput = np.repeat(layerOutput, bMult)
        inverseddEdInputs = nextdEdInputs.take(inverseConsols)
        dEdWs = expandedLayerOutput * inverseddEdInputs
        dEdBs = nextdEdInputs
        #upt to here correct
        def foo(x):
            if x >0.0:
                return 1.0
            else:
                return 0.0
        gtCast = np.vectorize(foo)
#         dOutsdIns = np.gt(layerOutput, np.zeros_like(layerOutput))
#         dOutsdIns = np.cast(dOutsdIns, 'float32')

        dOutsdIns = gtCast(layerOutput)
        dEdConnections = ws * inverseddEdInputs
        dEdConnectionGroups = dEdConnections.reshape((dEdConnections.size // bMult, bMult))
        dEdOuts = np.dot(dEdConnectionGroups, bMultOnes) #could need to be axis 1

        newdEdInputs = dEdOuts * dOutsdIns

        return (dEdWs, dEdBs, newdEdInputs)
gradLayer(dEdIns, prevOut)

(array([ 2,  4,  4,  6,  6, 10, 10,  8,  8, 12]),
 array([1, 2, 3, 4, 5, 6]),
 array([  3.,   5.,   8.,   9.,  10.]))

In [59]:
x = np.array(range(6))

In [60]:
x.reshape((2,3))

array([[0, 1, 2],
       [3, 4, 5]])

In [2]:
import numerai
trainingfeats, traininglabels, testingfeats, testinglabels = numerai.readTraining()

In [3]:
len(testingfeats)

10000

In [9]:
labels[0]

array([ 1.], dtype=float32)

In [1]:
%%bash
rm evolve.pyc
rm fulcrum.pyc
rm fractalNetwork2.pyc

In [31]:
import fulcrum
fulcrum.run()

AttributeError: 'module' object has no attribute 'run'

In [4]:
1 == 1.0

True